<a href="https://colab.research.google.com/github/aliebi/topic-modeling/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source of this code is for fastAi nlp cours

topic modeling 

In [0]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt

from sklearn.feature_extraction import stop_words

we select only some of them to compute from the whole dataset

In [0]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

In [0]:
newsgroups_train.filenames.shape, newsgroups_train.target.shape

((2034,), (2034,))

In [0]:
newsgroups_train.filenames

array(['/root/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38816',
       '/root/scikit_learn_data/20news_home/20news-bydate-train/talk.religion.misc/83741',
       '/root/scikit_learn_data/20news_home/20news-bydate-train/sci.space/61092',
       ...,
       '/root/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38737',
       '/root/scikit_learn_data/20news_home/20news-bydate-train/alt.atheism/53237',
       '/root/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38269'],
      dtype='<U86')

we have four topic 

In [0]:
newsgroups_train.target_names

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']

In [0]:
n=len(newsgroups_train.target_names)
print(n)

4


In [0]:
we vectorize each topic 

In [0]:
np.array(newsgroups_train.target_names)[newsgroups_train.target[:n]]

array(['comp.graphics', 'talk.religion.misc', 'sci.space', 'alt.atheism'],
      dtype='<U18')

In [0]:
newsgroups_train.target[:100]

array([1, 3, 2, 0, 2, 0, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 0, 2, 2, 3, 2, 1,
       3, 3, 1, 1, 2, 1, 2, 3, 0, 3, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 1,
       1, 0, 0, 3, 3, 3, 1, 2, 3, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1,
       2, 0, 1, 3, 1, 2, 2, 1, 2, 1, 0, 0, 2, 1, 3, 3, 0, 3, 2, 2, 0, 2,
       0, 3, 1, 3, 0, 2, 0, 3, 0, 3, 2, 2])

In [0]:
num_topics, num_top_words=6,8

In [0]:
sorted(list(stop_words.ENGLISH_STOP_WORDS))[:30]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere']

In [0]:
import nltk
nltk.download('wordnet')
from nltk import stem

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:

wnl = stem.WordNetLemmatizer()
porter = stem.porter.PorterStemmer()

In [0]:

word_list = ['feet', 'foot', 'foots', 'footing']
# word_list=['fly', 'flies', 'flying']
# word_list=['organize', 'organizes', 'organizing']
# word_list=['universe','university']

In [0]:

[wnl.lemmatize(word) for word in word_list]

['foot', 'foot', 'foot', 'footing']

In [0]:
[porter.stem(word) for word in word_list]

['feet', 'foot', 'foot', 'foot']

here we can see spacy lematizer that doesnt have stemmer 

In [0]:
import spacy
from spacy.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()
[lemmatizer.lookup(word) for word in word_list]

['feet', 'foot', 'foots', 'footing']

and also stopwords are different in sklearn and spacy you can compare them

In [0]:
nlp = spacy.load("en_core_web_sm")

sorted(list(nlp.Defaults.stop_words))[:30]

["'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(stop_words='english')

vectors = vectorizer.fit_transform(newsgroups_train.data).todense()
vectors.shape

(2034, 26576)

In [0]:
vectors[0]

matrix([[0, 0, 0, ..., 0, 0, 0]])

here we have (2034, 26576) matrix that we have 2034 document and 26576 uniqe words as featurs we collect

we have vocabulary of words by size11314

In [0]:

vocab = np.array(vectorizer.get_feature_names())
vocab.shape

(26576,)

In [0]:
%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

CPU times: user 1min 39s, sys: 6.28 s, total: 1min 45s
Wall time: 54 s


here 
U is each document per n topic 
s is the topic by topic model n*n 
Vh is n topic by vocab 

In [0]:
np.diag(Vh[:4])

array([-9.40971949e-03, -1.76916681e-02, -2.33939629e-05, -7.80408114e-05])

In [0]:
np.diag(U[:4])

array([-0.00477991, -0.00017499, -0.00083183,  0.00371775])

In [0]:
np.diag(s[:4])

array([[433.92698542,   0.        ,   0.        ,   0.        ],
       [  0.        , 291.51012741,   0.        ,   0.        ],
       [  0.        ,   0.        , 240.71137677,   0.        ],
       [  0.        ,   0.        ,   0.        , 220.00048043]])

now we can check is these matrics are true 

In [0]:
reconstructr_vect=U @ np.diag(s) @ Vh
np.allclose(reconstructr_vect,vectors)

True

In [0]:

num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [0]:

show_topics(Vh[:10])

['ditto critus propagandist surname galacticentric kindergarten surreal imaginative',
 'jpeg gif file color quality image jfif format',
 'graphics edu pub mail 128 3d ray ftp',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'god atheists atheism religious believe religion argument true',
 'space nasa lunar mars probe moon missions probes',
 'image probe surface lunar mars probes moon orbit',
 'argument fallacy conclusion example true ad argumentum premises',
 'space larson image theory universe physical nasa material']

in the next part we talk about 

In [0]:
m,n=vectors.shape
d=5  # num topics

clf = decomposition.NMF(n_components=d, random_state=1)

W1 = clf.fit_transform(vectors)
H1 = clf.components_
show_topics(H1)

['jpeg image gif file color images format quality',
 'edu graphics pub mail 128 ray ftp send',
 'space launch satellite nasa commercial satellites year market',
 'jesus god people matthew atheists does atheism said',
 'image data available software processing ftp edu analysis']

we create model by countvectorize and now we can make it by tf-idf 

In [0]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(newsgroups_train.data) # (documents, vocab)
vectors_tfidf

<2034x26576 sparse matrix of type '<class 'numpy.float64'>'
	with 133634 stored elements in Compressed Sparse Row format>

In [0]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_
show_topics(H1)

['people don think just like objective say morality',
 'graphics thanks files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief']